In [1]:
import numpy
from numpy import *
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib import *
import matplotlib as mpl
from scipy import ndimage
import sys, os
from IPython.display import clear_output

In [2]:
#DEFINING FUNCTIONS
# the authot of the BBXimportNorm function is Johannes Förster (Max Planck Institute for Intelligent Systems, Stuttgart, Germany)
def BBXimportNorm(M, path): #exporting dynamic magnetic contrast
    with open(path, 'rb') as f:    
        a = fromfile(f, dtype = '>u4')   
    N = a[0]
    Y = a[2]
    X = a[1]
    Bild = zeros([Y, X, N])
    for k in range(0, N): 
        for i in range(0, Y):
            for j in range(0, X):
                Bild[i, j, k * M % N]=a[3 + i + Y * j + Y * X * k]
    
    Normbild = zeros(shape(Bild))
    S = mean(Bild, axis=2)
    for i in range(0, N):
        Normbild[:, :, i] = flip(Bild[:, :, i] / S, axis = 0)
    del(Bild)
    return Normbild, N
    
def BBXimport(M, path): #exporting the chemical contrast
    with open(path, 'rb') as f:    
        a = fromfile(f, dtype = '>u4')   
    N = a[0]
    Y = a[2]
    X = a[1]
    Bild = zeros([Y, X, N])
    for k in range(0, N): 
        for i in range(0, Y):
            for j in range(0, X):
                Bild[i, j, k * M % N] = a[3 + i + Y * j + Y * X * k]
    
    Normbild = zeros(shape(Bild))
    S = mean(Bild, axis=2)
    for i in range(0, N):
        Normbild[:, :, i] = flip(Bild[:, :, i], axis = 0)
    del(Bild)
    return Normbild, N

In [20]:
#GETTING path from user input
import tkinter as tk
from tkinter import filedialog
import easygui as eg

fName = ''

root = tk.Tk()
root.withdraw()

path = filedialog.askdirectory() + '/'
print(path)

D:/Santa/Samples/StripeEasy/200924_BESSY_Oct20/STXM/Maxymus_Oct20/Santa/raw/201030/transient/M188-N7/


In [6]:
#modifying bunch of dirs with bbx-files
question = "Chemical or magnetic contrast?"
title = "Choose data type"
listOfOptions = ['chemical', 'magnetic']

choice = eg.choicebox(question , title, listOfOptions)
print(choice)

#PROCESSING ALL BBX-FILES in the path:
#path = "D:/Santa/Samples/T-stripes/181030b_BESSY_Jan-Apr19/STXM/0_DegRot/vertical/108mT/"

for fileDir in os.listdir(path):
    subDirName = os.path.join(path, fileDir) + '/'
    if os.path.isdir(subDirName):
        clear_output(wait=True)
        print(subDirName)
        for file in os.listdir(subDirName):
            if file.endswith(".bbx"):
                print(file)
                a = BBXimport(125, subDirName + file)
                data = zeros(shape(a[0]))

                for i in range (0,len(a[0])):
                    for j in range (0,len(a[0][i])):
                        for k in range (0,a[1]):
                            if choice == 'chemical':
                                y = a[0][i][j][k] #MODE1 raw data ("_%d (2).txt")
                            else:
                                y = a[0][i][j][k] / (sum(a[0][i][j]) / a[1]) #MODE2 magnetic contrast ("_%d.txt")

                            data[i, j, k] = y
#                            print(y)

                

                for k in range (0, a[1]):
                    print(k+1, '/', a[1])
                    if choice == 'chemical':
                        print(choice)
                        with open(subDirName + file[:-4] + "_%d (2).csv" % k,'w+') as f: #MODE1
                            for i in range (0, len(data)):
                                for j in range (0, len(data[i])):
                                    f.write(str(data[i, j, k]) + ',')
                                f.write('\n')
                    else:
                        print(choice)
                        with open(subDirName + file[:-4] + "_%d.csv" % k,'w+') as f: #MODE2
                            for i in range (0, len(data)):
                                for j in range (0, len(data[i])):
                                    f.write(str(data[i, j, k]) + ',')
                                f.write('\n')
    
                        f.close()
#    for dirname in dirs:
#        print(path + dirname)
            
print('done')

D:/Santa/Samples/StripeEasy/200924_BESSY_Oct20/STXM/Maxymus_Oct20/Santa/analysed/201030/Cplus/200mT/
done


In [22]:
question = "Chemical or magnetic contrast?"
title = "Choose data type"
listOfOptions = ['chemical', 'magnetic']

choice = eg.choicebox(question , title, listOfOptions)
print(choice)

#PROCESSING ALL BBX-FILES in the path:
#path = "D:/Santa/Samples/T-stripes/181030b_BESSY_Jan-Apr19/STXM/0_DegRot/vertical/108mT/"

fnames = []
for file in os.listdir(path):
    if file.endswith(".bbx"):
        fnames.append(os.path.join(file)[:-4])
print(fnames)

for fname in fnames:
    a = BBXimport(125, path + fname + '.bbx')
    data = zeros(shape(a[0]))

    for i in range (0,len(a[0])):
        for j in range (0,len(a[0][i])):
            for k in range (0,a[1]):
                if choice == 'chemical':
                    y = a[0][i][j][k] #MODE1 raw data ("_%d (2).txt")
                else:
                    y = a[0][i][j][k] / (sum(a[0][i][j]) / a[1]) #MODE2 magnetic contrast ("_%d.txt")

                data[i, j, k] = y

    print(y)

    for k in range (0, a[1]):
        clear_output(wait=True)
        print(k+1, '/', a[1])
        if choice == 'chemical':
            print(choice)
            with open(path + fname + "_%d (2).csv" % k,'w+') as f: #MODE1
                for i in range (0, len(data)):
                    for j in range (0, len(data[i])):
                        f.write(str(data[i, j, k]) + ',')
                    f.write('\n')
        else:
            print(choice)
            with open(path + fname + "_%d.csv" % k,'w+') as f: #MODE2
                for i in range (0, len(data)):
                    for j in range (0, len(data[i])):
                        f.write(str(data[i, j, k]) + ',')
                    f.write('\n')
    
            f.close()

print('done')

7 / 7
magnetic
done


In [5]:
#modified
question = "Chemical or magnetic contrast?"
title = "Choose data type"
listOfOptions = ['chemical', 'magnetic']

choice = eg.choicebox(question , title, listOfOptions)
print(choice)

#PROCESSING ALL BBX-FILES in the path:
for file in os.listdir(path):
    if file.endswith(".bbx"):
        print(file)
        a = BBXimport(125, path + file)
        data = zeros(shape(a[0]))

        for i in range (0,len(a[0])):
            for j in range (0,len(a[0][i])):
                for k in range (0,a[1]):
                    if choice == 'chemical':
                        y = a[0][i][j][k] #MODE1 raw data ("_%d (2).txt")
                    else:
                        y = a[0][i][j][k] / (sum(a[0][i][j]) / a[1]) #MODE2 magnetic contrast ("_%d.txt")

                    data[i, j, k] = y

        print(y)

        for k in range (0, a[1]):
            clear_output(wait=True)
            print(k+1, '/', a[1])
            if choice == 'chemical':
                print(choice)
                with open(path + file[:-4] + "_%d (2).csv" % k,'w+') as f: #MODE1
                    for i in range (0, len(data)):
                        for j in range (0, len(data[i])):
                            f.write(str(data[i, j, k]) + ',')
                        f.write('\n')
            else:
                print(choice)
                with open(path + file[:-4] + "_%d.csv" % k,'w+') as f: #MODE2
                    for i in range (0, len(data)):
                        for j in range (0, len(data[i])):
                            f.write(str(data[i, j, k]) + ',')
                        f.write('\n')
    
                f.close()

chemical
